#  Open Gust Map 

## Recupération des données via API Meteo france

In [ ]:
import geojson
import requests
import pandas as pd
import geopandas as gpd #panda géo
from geoalchemy2 import Geometry, WKTElement
from io import StringIO #récupération fichier
import time #calcul temps
import os 
from tqdm import tqdm #calcul temps requête
from dotenv import load_dotenv

In [ ]:
#Fichier liste des stations réseau étendue active (2100)
path='your_path' #chemin à la donnée
liste_stations= pd.read_csv(path, delimiter=',')

In [ ]:
# Ajouter un 0 en début de chaîne si la longueur est de 7 caractères
liste_stations['Id_station'] = liste_stations['Id_station'].astype(str).apply(lambda x: '0' + x if len(x) == 7 else x)

# Vérifier que toutes les cellules contiennent 8 caractères
all_cells_have_8_chars = all(liste_stations['Id_station'].apply(lambda x: len(x) == 8))

print("Toutes les cellules contiennent 8 caractères :", all_cells_have_8_chars)

In [ ]:
# Chargement du fichier .env
load_dotenv()

# attribut du fichier .env
apikey_commande = os.getenv("apikey_commande", None)
apikey_fichier = os.getenv("apikey_fichier", None)

In [ ]:
#Paramètres d'appel de requête
headers_commande = {
    #Type de sortie de la requête (dans le response header de MF)
    'accept': 'application/json',
    #Clé API
    'apikey' : apikey_commande }

In [ ]:
#Paramètres d'appel de requête
headers_commande = {
    #Type de sortie de la requête (dans le response header de MF)
    'accept': 'application/json',
    #Clé API
    'apikey' : apikey_fichier}

In [ ]:
# Filtre station ouest france
liste_dep=[29,22,56,35,44,53,72,49,85,17,79,86,16,87,23,36,18,45,41,37,28,50,14,61,76,27,91,78,93,94,95,95,75,92,60,80,62]
liste_stations['Dept'] = liste_stations.Id_station.astype('string').str.zfill(8).str[:2].astype(int)
df=liste_stations[liste_stations.Dept.isin(liste_dep)]

### Requête de commande

In [ ]:
# Initialiser les DataFrames
df_commandes = pd.DataFrame(columns=["id", "commande"])
df_commandes_erreur = pd.DataFrame(columns=["id", "commande"])
df_fichier_final = pd.DataFrame()
df_fichier_erreur = pd.DataFrame(columns=["commande"])

# Boucle principale
temps_total = 0
for id_station in tqdm(df['Id_station'],desc="Progression des requêtes"):
    start_time = time.time()  # Début du chrono
    
    # Récupérer le numéro de commande à l'aide de l'API
    url_commande = f"https://public-api.meteofrance.fr/public/DPClim/v1/commande-station/horaire?id-station={id_station}&date-deb-periode=2023-11-01T00:00:00Z&date-fin-periode=2023-11-02T23:00:00Z"
    commande = requests.get(url_commande, headers=headers_commande)
    
    if commande.status_code == 202:
        numero_commande = commande.json()['elaboreProduitAvecDemandeResponse']['return']

        # Ajouter l'ID de station et le numéro de commande au DataFrame df_commandes
        df_temp_commande = pd.DataFrame({"id": [id_station], "commande": [numero_commande]})
        df_temp_commande.to_csv('commande_hackaton.csv', mode='a', header=False)
        print(f"{id_station}, ok")
        time.sleep(2)
    else:
        #Données erreurs
        print(f"Erreur pour la station {id_station}, Code de status {commande.status_code}")
        temp_df_erreur = pd.DataFrame({"id": [id_station],"erreur": [commande.status_code]})
        time.sleep(2)
        
    end_time = time.time()  # Fin du chronomètre
    temps_total += (end_time - start_time)
    print(f"Temps écoulé pour la station {id_station}: {end_time - start_time:.2f} secondes")

### Requête Fichier

In [ ]:
# Lecture fichier des commandes
path='your_path' #chemin à la donnée
df_fichier=pd.read_csv(path, header=None)
df_fichier.columns = ['index','id','commande']

In [ ]:
temps_total=0

for num_commande in tqdm(df_fichier['commande'], desc='Progression des requêtes'):
    url = f"https://public-api.meteofrance.fr/public/DPClim/v1/commande/fichier?id-cmde={num_commande}"
    start_time = time.time()  # Début du chrono
    
    
    # Faire un appel d'API
    fichier = requests.get(url, headers=headers_fichier)

    if fichier.status_code == 201:
        # Traitement du fichier
        data = fichier.text.replace(',', '.')
        data_io = StringIO(data)
        temp_df = pd.read_csv(data_io, sep=';')

        # Sauvegarde des fichiers
        temp_df.to_csv('./data/data_hackaton.csv', mode='a', header=False)
        time.sleep(2)

        print(f"Récupération des données pour la commande {num_commande}")
    else:
        print(f"Erreur pour la commande {num_commande} , {fichier.status_code}")
        temp_df_erreur = pd.DataFrame({"commande": [num_commande]})
        time.sleep(2)


    end_time = time.time()  # Fin du chronomètre
    temps_total += (end_time - start_time)
    print(f"Temps écoulé pour le fichier {num_commande}: {end_time - start_time:.2f} secondes")

In [ ]:
#chemin à la donnée
path='your_path'
table = pd.read_csv(path)

In [ ]:
Ciaran = table.drop(table.columns[0], axis=1)
Ciaran.columns = temp_df.columns

## Data Prep

In [ ]:

# Recupération des coordonées des stations récupérés pour avoir le vent et les coordonées
table = pd.merge(Ciaran, liste_stations, left_on = "POSTE", right_on = "Id_station", how = 'left')

Table_work = table.groupby("POSTE",as_index=False).agg({"DXI":"first",
                                                               "FXI":"max",
                                                               "Altitude":'first',
                                                               'Longitude':'first',
                                                               'Latitude':'first'})

data = Table_work.copy()
data= data.dropna()

## Kringing

In [2]:
# Importing necessary libraries
import numpy as np
import pandas as pd
from pykrige.rk import Krige
from sklearn.model_selection import GridSearchCV, train_test_split
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from pykrige.ok3d import OrdinaryKriging3D
from pykrige.uk3d import UniversalKriging3D
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.interpolate import griddata
import rasterio

In [3]:
#Changement de CRS (pour être en 2154)
data_gdf = gpd.GeoDataFrame(data, geometry=gpd.points_from_xy(data.Longitude,data.Latitude), 
                              crs='EPSG:2154').to_crs("EPSG:4326")

# Extraction données géo
data_gdf['Longitude'] = data_gdf.geometry.x
data_gdf['Latitude'] = data_gdf.geometry.y

data = data_gdf.drop(columns = ['geometry']).copy()

In [5]:
# Step 1: Préparation des données
X = data[['Longitude', 'Latitude', 'Altitude']]
y = data[['FXI']]*3.6


min_latitude=min(data['Latitude'])
min_longitude=min(data['Longitude'])
max_longitude=max(data['Longitude'])
max_latitude=max(data['Latitude'])
min_altitude=min(data['Altitude'])
max_altitude=max(data['Altitude'])
num_points=1000


# Sélectionner les colonnes 
latitude = data['Latitude'].values
longitude = data['Longitude'].values
altitude = data['Altitude'].values
FXI = data['FXI'].values * 3.6 #passage en km/h

In [8]:
# Step 2: Création du modele Ordinary Kriging (général)
OK = OrdinaryKriging3D(longitude, latitude, altitude, FXI, variogram_model='linear',
                     verbose=False, enable_plotting=False)


In [10]:
# Step 4: Défibition de la grille de prédiction

longitude_range = np.linspace(min_longitude, max_longitude, num_points)
latitude_range = np.linspace(min_latitude, max_latitude, num_points)


In [ ]:
# Step 5: Import du raster relief

Raster = rasterio.open("./data/MNT_BD_ALTI_25m.tif")

# Adaptation de la grille avec ce dernier
prediction_points = np.array(np.meshgrid(longitude_range, latitude_range)).T.reshape(-1, 2)
coord_list = [(x, y) for x, y in prediction_points]

coord_list_alti = np.array([x[0] for x in Raster.sample(coord_list) if x != np.nan])

coord_list_alti_no_inf = np.where((coord_list_alti<5)&(coord_list_alti>-150), 5, coord_list_alti)
coord_list_alti_no_inf = np.where(coord_list_alti<-100, 0, coord_list_alti)

# Sauvegarde grille
with open('alit_1000.npy', 'wb') as f:
    np.save(f, coord_list_alti_no_inf)

In [12]:
# Step 6: Prédictions

with open('./data/alti_1000.npy', 'rb') as f:
    a = np.load(f)

predicted_FXI = OK.execute("points", prediction_points[:,0], prediction_points[:,1], a)


In [13]:
# Reshpahe prédictions pour la grille
k3d3 = predicted_FXI[0].reshape(num_points,num_points).T

In [ ]:
# Affichage du krigeage (carte de chaleur)
plt.figure(figsize=(20,20))
plt.imshow(k3d3, origin="lower", cmap='jet', interpolation='nearest')
plt.title("Krigeage 500m")
plt.xlabel("Long")
plt.ylabel("Lat")
plt.show()
